# Import Meta Reviewer Suggestions

### Neil D. Lawrence 13th April 2014

#### updated 2nd November 2015 in preparation for NIPS 2016

Get the meta reviewer names from the google spreadsheet. Then update the reviewer data base, `reviewer.db` with meta reviewers, then export as invites into CMT. Need to make a list of the meta reviewers only from the local data base to upload to CMT for the invites. Potential Area Chairs were stored in a Google doc spreadsheet. For access to the google spreadsheet to work, you need to have set up `oauth2` and shared the sheets for access. 

Configuration settings specific to `cmtutils.py` are stored in `.cmt_user.cfg`. Inside this file should be a section as follows:

```
[google docs]
# Here include the spreadsheet keys of program committee and reviewer candiates
program_committee_key = 
reviewer_candidate_key =  
buddy_pair_key = 
global_results_key = 
```

The Google sheet pointed to by `program_committee_key` contains one entry per potential meta reviewer in the form: 

```
Name | Institute | Subjects | Email | Answer
```


In [3]:
import cmtutils.cmtutils as cu
from cmtutils.config import config
import pods
# Download our meta-reviewer names from google doc.
spreadsheet_key = config.get('google docs', 'program_committee_key')
spreadsheet_name = '2014-01-19_NIPS_ACs'
res = pods.google.resource(id=spreadsheet_key)
ds = cu.drive_store(res, spreadsheet_name)
ds.read_meta_reviewers()


AttributeError: module 'pods.google' has no attribute 'resource'

Now take a look at the suggested area chairs to make sure they were read in correctly.

In [ ]:
ds

To try and keep track of everything we have a local SQLite data base. Here we load that data base in and add invites, labelling them as 'FirstRoundAreaChairInvite'.

In [15]:
# Load in the reviewer data base.
db = cu.reviewerdb('test.db')
fieldname = 'FirstRoundAreaChairInvite'
db.add_field(fieldname) # Add the field if it doesn't exist already

0

In [16]:
db

,ID,FirstName,MiddleNames,LastName,Institute,Email,ScholarID,Subjects,MSAcademicID,Webpage,...,IsReviewer,OtherEmails,IsMetaReviewer,PapersSince2012,NIPSDatabaseNominated2,NipsNominatedCorinnaNeilApproved,FromFullUserList,SecoundRoundInvite,FromAISTATS2012,FirstRoundAreaChairInvite
0,1,Aaditya,,Ramdas,CMU,aramdas@cs.cmu.edu,None,None,None,None,...,1,None,NaN,NaN,NaN,NaN,NaN,None,None,None
1,2,Aaron,,Courville,University of Montreal,aaron.courville@umontreal.ca,None,None,None,None,...,1,aaron.courville@gmail.com,NaN,1,NaN,NaN,NaN,None,None,None
2,3,Aaron,,Hertzmann,Adobe Research,hertzman@dgp.toronto.edu,None,None,None,None,...,1,hertzman@adobe.com,NaN,1,NaN,NaN,NaN,None,None,None
3,4,Aarti,,Singh,Carnegie Mellon,aartisingh@cmu.edu,None,None,None,None,...,NaN,None,NaN,4,NaN,NaN,NaN,None,None,None
4,5,Aasa,,Feragen,Max Planck Institutes T�bingen and University ...,aasa@di.ku.dk,None,None,None,None,...,1,aasa.feragen@tuebingen.mpg.de,NaN,1,NaN,NaN,NaN,None,None,None
5,6,Abdeslam,,Boularias,Max Planck Institute,boularias@tuebingen.mpg.de,None,None,None,None,...,1,None,NaN,2,NaN,NaN,NaN,None,None,None
6,7,Abe,,Friesen,,afriesen@cs.washington.edu,None,None,None,None,...,1,None,NaN,1,NaN,NaN,NaN,None,None,None
7,8,Abel,,Rodriguez,UC Santa Cruz,abel@soe.ucsc.edu,None,None,None,None,...,1,None,NaN,NaN,NaN,NaN,NaN,None,None,None
8,9,Abhimanyu,,Das,Microsoft Research,abhimand@usc.edu,None,None,None,None,...,NaN,abhidas@yahoo-inc.com,NaN,1,NaN,NaN,NaN,None,None,None
9,10,Abhinav,,Gupta,CMU,abhinavg@cs.cmu.edu,None,None,None,None,...,1,None,NaN,1,NaN,NaN,NaN,None,None,None


Now add the users to the data base.

In [ ]:
db.add_users(ds.reviewers, fieldname=fieldname)

We run an SQL command on the data base to count how many invites we have where there is no reply (we assume that the column Metareviewer is set to NULL when there is no reply).

In [ ]:
# This is number of non replied first round invitees.
print(db._execute_sql('SELECT COUNT(Active) FROM Reviewers WHERE ' + fieldname + '=1 AND Metareviewer is 0;'))
# This exports reviewers into a file called cmt_export.tsv ready for invite.
#db.export_reviewers(sql='FirstRoundInvite=1')

## Second Wave of Invites

We proceeded with a second wave of invites on March the 10th, this script loads those invites in, they are in the same spreadsheet file, but on a different sheet.

In [ ]:
spreadsheet_name = '2014-03-10_NIPS_ACs'
res = pods.google.resource(id=spreadsheet_key)
ds = cu.drive_store(res, spreadsheet_name)
ds.read_meta_reviewers()
fieldname='SecoundRoundAreaChairInvite'

Once again we can check users have been loaded from the spreadsheet properly.

In [ ]:
ds.reviewers

And now we can add them to the data base.

In [ ]:
db.add_users(ds.reviewers, fieldname=fieldname)

We can check how many reviewers we've just added who are not currently meta reviewers with the following SQL command.

In [ ]:
print(db._execute_sql("""SELECT COUNT(Active) FROM Reviewers WHERE ' + fieldname + '=1 AND Metareviewer is NULL;"""))
db.export_reviewers(sql=fieldname + '=1') # this exports those reviews into CMT for an upload.

### Counting Number of Metareviewers

This command gives us the total number of meta reviewers by calling SQL on the data base.

In [ ]:
db._execute_sql("""SELECT COUNT(Active) FROM Reviewers WHERE Metareviewer=1;""")